In [1]:

%run ./data.ipynb
%run ./model.ipynb
%run ./loss.ipynb
%run ./utils.ipynb

import os, time
from operator import add
import numpy as np
from glob import glob
import cv2
from tqdm import tqdm
import imageio
import torch
from sklearn.metrics import accuracy_score, f1_score, jaccard_score, precision_score, recall_score



def calculate_metrics(y_true, y_pred):
    """ Ground truth """
    y_true = y_true.cpu().numpy()
    y_true = y_true > 0.5
    y_true = y_true.astype(np.uint8)
    y_true = y_true.reshape(-1)

    """ Prediction """
    y_pred = y_pred.cpu().numpy()
    y_pred = y_pred > 0.5
    y_pred = y_pred.astype(np.uint8)
    y_pred = y_pred.reshape(-1)

    score_jaccard = jaccard_score(y_true, y_pred)
    score_f1 = f1_score(y_true, y_pred)
    score_recall = recall_score(y_true, y_pred)
    score_precision = precision_score(y_true, y_pred)
    score_acc = accuracy_score(y_true, y_pred)

    return [score_jaccard, score_f1, score_recall, score_precision, score_acc]

def mask_parse(mask):
    mask = np.expand_dims(mask, axis=-1)    ## (512, 512, 1)
    mask = np.concatenate([mask, mask, mask], axis=-1)  ## (512, 512, 3)
    return mask

if __name__ == "__main__":
    """ Seeding """
    seeding(42)

    """ Folders """
    create_dir("results")

    """ Load dataset """
    test_x = sorted(glob("./Data1/test/image/*"))
    test_y = sorted(glob("./Data1/test/mask/*"))

    """ Hyperparameters """
    H = 512
    W = 512
    size = (W, H)
    checkpoint_path = "files/checkpoint.pth"

    """ Load the checkpoint """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    model = attention_unet().to(device)
    
    model.load_state_dict(torch.load(checkpoint_path, map_location=device))
    model.eval()

    metrics_score = [0.0, 0.0, 0.0, 0.0, 0.0]
    time_taken = []

    for i, (x, y) in tqdm(enumerate(zip(test_x, test_y)), total=len(test_x)):
        """ Extract the name """
        name = x.split("/")[-1].split(".")[0]

        """ Reading image """
        image = cv2.imread(x, cv2.IMREAD_COLOR) ## (512, 512, 3)
        ## image = cv2.resize(image, size)
        x = np.transpose(image, (2, 0, 1))      ## (3, 512, 512)
        x = x/255.0
        x = np.expand_dims(x, axis=0)           ## (1, 3, 512, 512)
        x = x.astype(np.float32)
        x = torch.from_numpy(x)
        x = x.to(device)

        """ Reading mask """
        mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
        ## mask = cv2.resize(mask, size)
        y = np.expand_dims(mask, axis=0)            ## (1, 512, 512)
        y = y/255.0
        y = np.expand_dims(y, axis=0)               ## (1, 1, 512, 512)
        y = y.astype(np.float32)
        y = torch.from_numpy(y)
        y = y.to(device)

        with torch.no_grad():
            """ Prediction and Calculating FPS """
            start_time = time.time()
            pred_y = model(x)
            pred_y = torch.sigmoid(pred_y)
            total_time = time.time() - start_time
            time_taken.append(total_time)


            score = calculate_metrics(y, pred_y)
            metrics_score = list(map(add, metrics_score, score))
            pred_y = pred_y[0].cpu().numpy()        ## (1, 512, 512)
            pred_y = np.squeeze(pred_y, axis=0)     ## (512, 512)
            pred_y = pred_y > 0.5
            pred_y = np.array(pred_y, dtype=np.uint8)

        """ Saving masks """
        ori_mask = mask_parse(mask)
        pred_y = mask_parse(pred_y)
        line = np.ones((size[1], 10, 3)) * 128

        cat_images = np.concatenate(
            [image, line, ori_mask, line, pred_y * 255], axis=1
        )
        # Getting the current working directory
        current_directory = os.getcwd()

        result_dir = os.path.join(current_directory, "results")

# Check and create the directory if it doesn't exist
        if not os.path.exists(result_dir):
         os.makedirs(result_dir)



        save_path = os.path.join(result_dir, f"{name}.png")
        print("Saving image to:", save_path)

        success = cv2.imwrite(save_path, cat_images)

        if success:
            
            print("Image saved successfully!")
        else:
            print("Failed to save the image.")
       
        
        
        
        

    jaccard = metrics_score[0]/len(test_x)
    f1 = metrics_score[1]/len(test_x)
    recall = metrics_score[2]/len(test_x)
    precision = metrics_score[3]/len(test_x)
    acc = metrics_score[4]/len(test_x)
    print(f"Jaccard: {jaccard:1.4f} - F1: {f1:1.4f} - Recall: {recall:1.4f} - Precision: {precision:1.4f} - Acc: {acc:1.4f}")

    fps = 1/np.mean(time_taken)
    print("FPS: ", fps)

['D:\\deeplearning_project\\retina vessel segmentation', 'C:\\Users\\krish\\anaconda3\\python39.zip', 'C:\\Users\\krish\\anaconda3\\DLLs', 'C:\\Users\\krish\\anaconda3\\lib', 'C:\\Users\\krish\\anaconda3', '', 'C:\\Users\\krish\\anaconda3\\lib\\site-packages', 'C:\\Users\\krish\\anaconda3\\lib\\site-packages\\locket-0.2.1-py3.9.egg', 'C:\\Users\\krish\\anaconda3\\lib\\site-packages\\win32', 'C:\\Users\\krish\\anaconda3\\lib\\site-packages\\win32\\lib', 'C:\\Users\\krish\\anaconda3\\lib\\site-packages\\Pythonwin', 'C:\\Users\\krish\\anaconda3\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\krish\\.ipython']
torch.Size([8, 64, 128, 128])
torch.Size([8, 64, 128, 128])
torch.Size([8, 32, 256, 256])
torch.Size([8, 32, 256, 256])
torch.Size([8, 1, 256, 256])


  0%|                                                                                           | 0/20 [00:00<?, ?it/s]

torch.Size([1, 64, 256, 256])
torch.Size([1, 64, 256, 256])
torch.Size([1, 32, 512, 512])
torch.Size([1, 32, 512, 512])


  5%|████▏                                                                              | 1/20 [00:03<00:59,  3.13s/it]

Saving image to: D:\deeplearning_project\retina vessel segmentation\results\image\0.png
Image saved successfully!
torch.Size([1, 64, 256, 256])
torch.Size([1, 64, 256, 256])
torch.Size([1, 32, 512, 512])
torch.Size([1, 32, 512, 512])


...

Saving image to: D:\deeplearning_project\retina vessel segmentation\results\image\7.png
Image saved successfully!
torch.Size([1, 64, 256, 256])
torch.Size([1, 64, 256, 256])
torch.Size([1, 32, 512, 512])
torch.Size([1, 32, 512, 512])


 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [00:54<00:02,  2.74s/it]

Saving image to: D:\deeplearning_project\retina vessel segmentation\results\image\8.png
Image saved successfully!
torch.Size([1, 64, 256, 256])
torch.Size([1, 64, 256, 256])
torch.Size([1, 32, 512, 512])
torch.Size([1, 32, 512, 512])


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:57<00:00,  2.89s/it]

Saving image to: D:\deeplearning_project\retina vessel segmentation\results\image\9.png
Image saved successfully!
Jaccard: 0.6297 - F1: 0.7718 - Recall: 0.7356 - Precision: 0.8228 - Acc: 0.9625
FPS:  0.4344287686881136
